# Import library


In [354]:
%load_ext autoreload
%autoreload 2

# ===============================
# 1. Import 핵심 라이브러리 (우선순위: numerical/scientific/data)
import numpy as np           # 수치 해석용 (필수)
import pandas as pd          # 데이터프레임 처리용
import math                  # 수학 함수
from tqdm import tqdm        # 진행 상황 시각화
import matplotlib.pyplot as plt           # 플롯
import matplotlib.ticker as ticker        # 플롯 축 설정
import CoolProp.CoolProp as CP            # 열역학/물성치 구할 때 필요

# 2. 커스텀 라이브러리 (src 폴더 내)
import sys
sys.path.append('src')       # 커스텀 모듈 경로 추가

import dartwork_mpl as dm        # 플롯 스타일 커스텀
import enex_analysis as enex       # 모델 엔진 (src/dhw_ex_model)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 0. 준비


In [345]:
## Fontsize 지정
plt.rcParams['font.size'] = 9

fs = {
    'label': dm.fs(0),
    'tick': dm.fs(-1.0),
    'legend': dm.fs(-1.5),
    'subtitle': dm.fs(-0.5),
    'cbar_tick': dm.fs(-1.5),
    'cbar_label': dm.fs(-1.5),
    'cbar_title': dm.fs(-1),
    'setpoint': dm.fs(-1),
    'text': dm.fs(-2.0),
            }

pad = {
    'label': 6,
    'tick': 5,
}

LW = np.arange(0.25, 3.0, 0.25)


# 1. Constant


In [346]:
c_w = 4186  # J/kgK
rho_w = 1000  # kg/m3


# 2. 시뮬레이션 설정


## 2.1 Water use schedule


In [347]:
entries1 = [
        # --- 0:00 - 6:00 (사용 0) ---
        # --- 아침 피크 ---
        # ("0:00", "1:00", 0.5),
        ("6:00", "6:30", 0.5),
        ("6:30", "7:00", 0.9),
        ("7:00", "7:30", 1.0),  # <--- Peak (1.0)
        ("7:30", "8:00", 0.9),
        ("8:00", "8:30", 0.6),
        ("8:30", "9:00", 0.3),
        
        # --- 주간 (최소 0.05, 점심 피크) ---
        ("9:00", "11:30", 0.05), # 2.5시간 (5개 구간)
        ("11:30", "12:00", 0.2), # 점심 준비/손 씻기
        ("12:00", "12:30", 0.4), # 식사/설거지
        ("12:30", "13:00", 0.4), # 설거지/양치
        ("13:00", "13:30", 0.2), # 마무리
        ("13:30", "17:00", 0.05),# 3.5시간 (7개 구간)
        
        # --- 저녁 피크 ---
        ("17:00", "17:30", 0.3),
        ("17:30", "18:00", 0.5),
        ("18:00", "18:30", 0.8),
        ("18:30", "19:00", 0.9),
        ("19:00", "19:30", 0.8),
        ("19:30", "20:00", 0.7),
        
        # --- 저녁 (감소) ---
        ("20:00", "20:30", 0.5),
        ("20:30", "21:00", 0.4),
        ("21:00", "21:30", 0.3),
        ("21:30", "22:00", 0.2),
        # --- 22:00 - 24:00 (사용 0) ---
    ]

## 2.3 Outdoor air temperature schedule


In [348]:
# 실외 공기 온도 스케줄 생성
sim_period_sec = 72* enex.h2s
dt = 60
time = np.arange(0, sim_period_sec, dt)

T_oa_base = 20  # 최저 온도 [°C]
T_oa_amplitude = 5.0  # 온도 진폭 [°C]
# 진폭의 최대가 12시간(43200초) 이후에 발생하도록 평행이동 (위상 이동: -12시간)
T_oa_schedule = T_oa_base + T_oa_amplitude * np.sin(2 * np.pi * (time - 6 * enex.h2s) / (24 * enex.h2s))


In [314]:
# 2. 모델 객체 생성 (모든 설정값을 __init__에 전달)
# 정풍량 모델: dV_ou_design으로 고정 풍량 설정 (GSHPB의 정유량과 유사)
ashpb_model = enex.AirSourceHeatPumpBoiler(
    # 냉매/사이클/압축기 파라미터
    ref='R410A',
    V_disp_cmp=0.00005,
    eta_cmp_isen=0.7,
    
    # 열교환기 파라미터 (상수 UA)
    UA_cond_design=1000.0,  # 응축기 열전달 계수 [W/K] (상수)
    UA_evap_design=1000.0,  # 증발기 열전달 계수 [W/K] (상수)
    A_cross_ou=np.pi * 0.4 ** 2,         # 실외기 단면적 [m²]
    
    # 실외기 팬 파라미터 (정풍량)
    dV_ou_design=2,   # 실외기 설계 풍량 [m³/s] (정풍량)
    dP_ou_design=500.0, # 실외기 설계 정압 [Pa]
    eta_motor_ou=0.8,
    eta_fan_ou=0.8,
    
    # 탱크/제어/부하 파라미터
    T0=0.0,                    # 기준 외기 온도 [°C]
    T_tank_w_setpoint=65.0,    # 저탕조 설정 온도 [°C]
    T_tank_w_lower_bound=55.0, # 저탕조 하한 온도 [°C]
    T_serv_w=40.0,             # 서비스 급탕 온도 [°C]
    T_sup_w=15.0,              # 급수(상수도) 온도 [°C]
    heater_capacity=8000.0,    # 히터 최대 용량 [W]
    dV_w_serv_m3s=0.0001,      # 최대 급탕 유량 [m³/s]
    
    # 탱크 물성
    r0=0.2,      # 탱크 반지름 [m]
    H=0.8,       # 탱크 높이 [m]
    x_shell=0.01, # 탱크 외벽 두께 [m]
    x_ins=0.05,   # 단열재 두께 [m]
    k_shell=25,   # 탱크 외벽 열전도도 [W/mK]
    k_ins=0.03,   # 단열재 열전도도 [W/mK]
    h_o=15,       # 외부 대류 열전달계수 [W/m²K]
)


# 3. Simulation 실행 및 csv 저장


In [315]:
# 4. 시뮬레이션 실행
sim_results_df = ashpb_model.analyze_dynamic(
    simulation_period_sec=sim_period_sec, 
    dt_s=dt,                         
    T_tank_w_init_C=55.0,
    schedule_entries=entries1,
    T_oa_schedule=T_oa_schedule,  # 실외 공기 온도 스케줄 추가
    heater_capacity_const=8000.0,  # 히터 최대 용량 [W]
    result_save_csv_path='../result/ashpb_simulation_results.csv',
    # save_ph_diagram=True,
    # snapshot_save_path='./video'
)

ASHPB Simulating:  43%|████▎     | 1878/4320 [00:30<00:37, 65.19it/s] 

경고: 타임스텝 1902에서 최적화 실패, OFF 상태 결과 사용
입력 조건:
  T_tank_w: 37.75 [°C]
  T_oa: 22.15 [°C]
  Q_cond_load: 8000.00 [W]

최적화 결과 정보:
  메시지: Positive directional derivative for linesearch
  반복 횟수: 28
  최종 목적 함수 값 (E_tot): 1847.32 [W]
  초기 추정값: dT_ref_evap=5.00 [K], dT_ref_cond=20.00 [K]
  최종 변수 값: dT_ref_evap=1.0600 [K], dT_ref_cond=21.1279 [K]

제약 조건 위반 여부:
  응축기 LMTD 제약 조건 (Q_LMTD_cond - Q_cond_load): -0.00 [W]
    → 제약 조건 만족 (오차 < 1.0 W)
  증발기 LMTD 제약 조건 (Q_LMTD_evap - Q_ref_evap): 0.00 [W]
    → 제약 조건 만족 (오차 < 1.0 W)

최종 변수 값에서의 성능:
  E_tot: 1847.32 [W]
  E_cmp: 1644.03 [W]
  E_fan_ou: 203.29 [W]
  Q_LMTD_cond: 8000.00 [W]
  Q_LMTD_evap: 6355.97 [W]
  Q_ref_evap: 6355.97 [W]
  converged: True


ASHPB Simulating: 100%|██████████| 4320/4320 [01:19<00:00, 54.33it/s] 


In [316]:
df = pd.read_csv('../result/ashpb_simulation_results.csv')
false_converged_indices = df.index[df['converged'] == False].tolist()
false_converged_indices
# len(false_converged_indices)
df[df['converged']==False]

,Unnamed: 0,is_on,converged,Q_cond_load,Q_ref_cond,Q_ref_evap,Q_LMTD_cond,Q_LMTD_evap,E_cmp,E_fan_ou,...,h3,h4,s1,s2,s3,s4,x1,x2,x3,x4
1902,1902,True,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,262179.323388,262179.323388,1767.984096,1767.984096,1208.059141,1208.059141,0.0,0.0,0.0,0.0


# 4. Visualization


## 4.1 Data import


In [317]:
df = pd.read_csv('../result/ashpb_simulation_results.csv', index_col=0)
df.head()

,is_on,converged,Q_cond_load,Q_ref_cond,Q_ref_evap,Q_LMTD_cond,Q_LMTD_evap,E_cmp,E_fan_ou,E_tot,...,h3,h4,s1,s2,s3,s4,x1,x2,x3,x4
0,True,True,8000.0,8000.0,4996.805574,11614.572557,4996.805574,3003.194426,102.401338,3105.595765,...,298890.607547,298890.607547,1828.608038,1887.605560,1318.556053,1376.092674,52425.984312,108116.490897,72274.225662,56558.097698
1,True,True,8000.0,8000.0,5301.937945,10308.569796,5301.937945,2698.062055,243.921418,2941.983472,...,301519.690279,301519.690279,1812.580251,1863.106202,1326.208948,1373.201873,59512.567984,106549.278333,72812.920047,59976.802571
2,True,True,8000.0,8000.0,4896.937265,11676.933923,4896.937265,3103.062735,105.756488,3208.819223,...,304218.340836,304218.340836,1826.772129,1885.943921,1334.035081,1394.694738,53250.796752,109623.476058,73373.862346,56804.677006
3,True,True,8000.0,8000.0,5370.073885,9791.072045,5370.073885,2629.926115,483.419152,3113.345267,...,306997.045592,306997.045592,1803.824678,1850.713378,1342.064197,1386.622342,63239.950758,106953.099298,73959.414091,61788.356766
4,True,True,8000.0,8000.0,4832.670999,11566.230174,4832.670999,3167.329001,120.214428,3287.543429,...,309869.163807,309869.163807,1822.991357,1881.213774,1350.334259,1412.681181,54940.167798,110785.820427,74572.564842,57542.503214


In [318]:
# cop이 0이 아닌 값들만 추출하는 구문
# df.loc[조건, '컬럼명'] 형식으로, 조건을 만족하는 행들에서 'cop' 컬럼만 선택한다.
# 여기서 df['cop'] != 0은 cop 값이 0이 아닌 행을 필터링한다.
# 즉, cop_nonzero에는 cop이 0이 아닌 값들만 Series 형태로 저장된다.

cop_nonzero = df.loc[df['cop'] != 0, 'cop']

# [DataFrame의 loc 기능 상세 설명]
# df.loc[행_조건, 열_선택] 구조로,
# • 행_조건: 특정 조건(불리언 마스킹/인덱스/슬라이스 등)이나 행 라벨명을 지정할 수 있음
# • 열_선택: 하나 혹은 여러 컬럼명을 리스트로 지정 가능
# 예)
#   df.loc[5, 'cop']        # 인덱스가 5인 행의 'cop' 값
#   df.loc[[1,3,5], ['A','B']]  # 1,3,5번째 행에서 'A','B' 컬럼 추출
#   df.loc[df['A']>10, 'B'] # 'A'가 10보다 큰 행의 'B'컬럼
cop_min = cop_nonzero.min()
cop_max = cop_nonzero.max()
print(f"cop min (nonzero): {cop_min}")
print(f"cop max (nonzero): {cop_max}")

cop min (nonzero): 2.0499826703493595
cop max (nonzero): 4.598459021053218


## 4.2 Data extraction


In [319]:
# 시간 배열
time = df.index.to_numpy() * dt  # 초 단위

# 온도 데이터
T_tank_w = df['T_tank_w'].to_numpy()
T_oa = df['T_oa'].to_numpy()  # 실외 공기 온도
T_air_ou_out = df['T_air_ou_out'].to_numpy()
T0 = df['T0'].to_numpy()
T1 = df['T1'].to_numpy()
T2 = df['T2'].to_numpy()
T3 = df['T3'].to_numpy()
T4 = df['T4'].to_numpy()
T_cond = df['T_cond'].to_numpy()
T_evap = df['T_evap'].to_numpy()

# 열량 데이터
Q_ref_cond = df['Q_ref_cond'].to_numpy()
Q_ref_evap = df['Q_ref_evap'].to_numpy()
Q_cond_load = df['Q_cond_load'].to_numpy()

# 전력 데이터
E_cmp = df['E_cmp'].to_numpy()
E_fan_ou = df['E_fan_ou'].to_numpy()  # 실외기 팬 전력
E_tot = df['E_tot'].to_numpy()

# COP
COP = df['cop'].to_numpy()

# 팬 풍량
dV_fan_ou = df['dV_fan_ou'].to_numpy()  # 실외기 팬 풍량

# 유량 데이터
dV_w_serv = df['dV_w_serv'].to_numpy()
dV_w_sup_tank = df['dV_w_sup_tank'].to_numpy()
dV_w_sup_mix = df['dV_w_sup_mix'].to_numpy()

# 압력 데이터
P1 = df['P1'].to_numpy()
P2 = df['P2'].to_numpy()
P3 = df['P3'].to_numpy()
P4 = df['P4'].to_numpy()

# 엔탈피 데이터
h1 = df['h1'].to_numpy()
h2 = df['h2'].to_numpy()
h3 = df['h3'].to_numpy()
h4 = df['h4'].to_numpy()

# 엑서지 데이터
x1 = df['x1'].to_numpy()
x2 = df['x2'].to_numpy()
x3 = df['x3'].to_numpy()
x4 = df['x4'].to_numpy()

# 온도 단위 변환 (켈빈 → 섭씨)
T0_K = enex.C2K(T0)
T_tank_w_K = enex.C2K(T_tank_w)
T_oa_K = enex.C2K(T_oa)
T_serv_w_K = enex.C2K(df['T_serv_w'].to_numpy())
T_sup_w_K = enex.C2K(df['T_sup_w'].to_numpy())


In [320]:
T0_K

array([273.15, 273.15, 273.15, ..., 273.15, 273.15, 273.15], shape=(4320,))

In [321]:
# 엑서지 분석 (지중열 교환 관련 제거, 실외 공기 관련 추가)

# --- 1. 엑서지 입력/출력 항 계산 ---
X_cmp = E_cmp  # 압축기 엑서지 입력
X1 = x1  # State 1 엑서지
X2 = x2  # State 2 엑서지
X3 = x3  # State 3 엑서지
X4 = x4  # State 4 엑서지

# 실외 공기 엑서지 (증발기에서 흡수)
# 공기는 실외 온도에서 열을 잃음 (Q_ref_evap < 0)
X_air_evap = (1 - T_oa_K / T_oa_K) * Q_ref_evap  # 실외 공기 엑서지 변화량

# 응축기 엑서지 (냉매→저탕조)
X_ref_cond = (1 - T_oa_K / T_tank_w_K) * Q_ref_cond  # 응축기(냉매→저탕조) 엑서지 유입량

# 탱크 및 믹싱밸브 엑서지
X_tank_sup_w = enex.calc_exergy_flow(G=c_w * rho_w * dV_w_sup_tank, T=T_sup_w_K, T0=T_oa_K)  # 탱크 공급수 엑서지
X_tank_w = enex.calc_exergy_flow(G=c_w * rho_w * dV_w_sup_tank, T=T_tank_w_K, T0=T_oa_K)  # 탱크 온수부 엑서지
X_mix_sup_w = enex.calc_exergy_flow(G=c_w * rho_w * dV_w_sup_mix, T=T_sup_w_K, T0=T_oa_K)  # 믹싱밸브 공급수 엑서지
X_mix_serv_w = enex.calc_exergy_flow(G=c_w * rho_w * dV_w_serv, T=T_serv_w_K, T0=T_oa_K)  # 믹싱밸브 서비스 엑서지

# --- 2. 엑서지 생성/전달/손실 항 계산 ---
# 증발기 엑서지 손실 (실외 공기 → 냉매)
Xc_evap = (X_air_evap + X4) - X1  # 증발기 전후 손실
Xc_cmp = (X1 + X_cmp) - X2  # 압축기 통과 후 손실
Xc_ref_cond = X2 - X3 - X_ref_cond  # 응축기에서의 손실
Xc_exp = X3 - X4  # 팽창밸브 손실

Xc_tank = X_ref_cond + X_tank_sup_w - X_tank_w  # 탱크 전후 손실
Xc_mix = (X_tank_w + X_mix_sup_w - X_mix_serv_w)  # 믹싱부 전체 엑서지(총계)

# 순환흐름(유량 기반 exergy flow) 변수들 정의
X_flow_X1_to_X2 = X1 + X_cmp
X_flow_X2_to_X3 = X_flow_X1_to_X2 - X_ref_cond - Xc_ref_cond
X_flow_X3_to_X4 = X_flow_X2_to_X3 - Xc_exp
X_flow_X4_to_X1 = X_flow_X3_to_X4 + X_air_evap - Xc_evap


In [322]:
# =============================================================================
# 5) (옵션) 플롯 시작부
#    - 한글 폰트 설정 등은 사용 OS 환경에 맞게 수동 설정
# =============================================================================
def plot_simple_graph(df_column, time, xlabel, ylabel, xmin=0, xmax=24, Kelvin=False, color='oc.blue5', savepath=None):
    # 온도 데이터를 섭씨(°C)로 변환하여 플로팅
    if Kelvin:
        y = enex.K2C(df_column)
    else:
        y = df_column
    x = time * enex.s2h  # 초를 시간으로 변환

    fig, ax = plt.subplots(figsize=(dm.cm2in(16), dm.cm2in(6)))
    
    # --- 마이너 틱 설정 ---
    # X축: 주 틱 사이를 5개의 간격으로 나눔 (마이너 틱 4개 생성)
    ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
    # Y축: 주 틱 사이를 2개의 간격으로 나눔 (마이너 틱 1개 생성)
    ax.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))

    ax.plot(x, y, color=color, linewidth=0.5,)
    ax.set_xlim(xmin, xmax)
    if xmax % 24 == 0: 
        if xmax == 24:
            ax.set_xticks(np.arange(0, 25, 2))
        elif xmax == 48:
            ax.set_xticks(np.arange(0, xmax+1, 6))
        elif xmax == 72:
            ax.set_xticks(np.arange(0, xmax+1, 12))
        elif xmax > 72:
            ax.set_xticks(np.arange(0, xmax+1, 24))

    ax.tick_params(axis='both', which='both', labelsize=fs['tick'], pad=pad['tick'])
    ax.set_ylim(np.nanmin(y)*0.9, np.nanmax(y)*1.1)
    ax.set_xlabel(xlabel, fontsize=fs['label'], labelpad=pad['label'])
    ax.set_ylabel(ylabel, fontsize=fs['label'], labelpad=pad['label'])
    dm.simple_layout(fig, margins=(0.05, 0.05, 0.05, 0.05), bbox=[0, 1, 0.02, 1])
    if savepath:
        plt.savefig(f'{savepath}.png')
    dm.save_and_show(fig)
    plt.close()

def plot_multi_graph(df_columns, legends, time, xlabel, ylabel, linestyles=None, colors=None, xmin=0, xmax=24, Kelvin=False, savepath=None, scatter=False):
    """
    여러 개의 데이터 열을 받아 하나의 그래프에 플로팅하는 함수.
    scatter=True이면 점 그래프(scatter plot)를 그립니다.
    """
    if len(df_columns) != len(legends):
        raise ValueError("데이터 컬럼의 개수와 레이블의 개수가 일치해야 합니다.")

    fig, ax = plt.subplots(figsize=(dm.cm2in(16), dm.cm2in(6)))
    x = time * enex.s2h  # 초를 시간으로 변환
    
    if colors is None:
        colors = ['oc.blue5', 'oc.orange5', 'oc.green5', 'oc.red5', 'oc.violet5',
                  'oc.gray5', 'oc.yellow5', 'oc.cyan5']
    
    global_min = np.inf
    global_max = -np.inf
    processed_ys = []
    for col in df_columns:
        y = enex.K2C(col) if Kelvin else col
        processed_ys.append(y)
        if not y.empty if hasattr(y, 'empty') else len(y) > 0:
            global_min = min(global_min, np.nanmin(y))
            global_max = max(global_max, np.nanmax(y))
    
    for i, y_data in enumerate(processed_ys):
        current_color = colors[i % len(colors)]
        current_label = legends[i]

        if scatter:
            ax.scatter(x, y_data, color=current_color, label=current_label, s=1.5, alpha=0.3)
        else:
            current_linestyle = linestyles[i % len(linestyles)] if linestyles is not None else ['-', '--', '-.', ':'][i % 4]
            ax.plot(x, y_data, color=current_color, linewidth=0.8, label=current_label, linestyle=current_linestyle)

    ax.set_xlim(xmin, xmax)
    if xmax % 24 == 0: 
        if xmax == 24:
            ax.set_xticks(np.arange(0, 25, 2))
        elif xmax == 48:
            ax.set_xticks(np.arange(0, xmax+1, 6))
        elif xmax == 72:
            ax.set_xticks(np.arange(0, xmax+1, 12))
        elif xmax > 72:
            ax.set_xticks(np.arange(0, xmax+1, 24))
    
    ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
    ax.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))
    
    ax.tick_params(axis='both', which='both', labelsize=fs['tick'], pad=pad['tick'])
    
    if np.isinf(global_min) or np.isinf(global_max):
        ax.set_ylim(0, 1)
    else:
        global_int = global_max - global_min
        ax.set_ylim(global_min - global_int * 0.3, global_max + global_int * 0.3)
    
    ax.set_xlabel(xlabel, fontsize=fs['label'], labelpad=pad['label'])
    ax.set_ylabel(ylabel, fontsize=fs['label'], labelpad=pad['label'])

    ax.legend(ncol=6, fontsize=fs['legend'])
    dm.simple_layout(fig, margins=(0.05, 0.05, 0.05, 0.05), bbox=[0, 1, 0.02, 1])
    
    if savepath:
        plt.savefig(f'{savepath}.png')
        
    dm.save_and_show(fig)
    plt.close()


## 5.1 Hot water tank temp and water schedule


In [349]:
# ===================== 사용자 입력(수정 지점) =====================
X = time * enex.s2h  # 시간 [h]
Y1 = df['dV_w_serv'] * enex.m32L / enex.s2m  # 온수 사용량 [L/min]

X_LABEL = 'Hour of day [h]'
Y_LABEL = 'Hot water usage [L/min]'

xmin1, xmax1, xint1, xmar1 = 0, 24, 4, 0
ymin1, ymax1, yint1, ymar1 = 0, 8, 2, 0
color_ax1 = 'oc.red'
# ===============================================================

fig, ax1 = plt.subplots(1,1, figsize=(dm.cm2in(10), dm.cm2in(5.5)))

ax1.plot(X, Y1, label='', linewidth=LW[2], color=color_ax1 + '4')

ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y_LABEL, fontsize=fs['label'], labelpad=pad['label'])

ax1.set_xlim(xmin1 - xmar1, xmax1 + xmar1)
ax1.set_ylim(ymin1 - 0.2, ymax1 + ymar1)

ax1.set_xticks(np.arange(xmin1, xmax1*1.001, xint1))
ax1.set_yticks(np.arange(ymin1, ymax1*1.001, yint1))

ax1.tick_params(labelsize=fs['tick'], which='major', length=2.5, width=0.3, pad=pad['tick'])
ax1.tick_params(labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])

ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

ax1.tick_params(axis='y', labelsize=fs['tick'], which='major', pad=pad['tick'],)
ax1.tick_params(axis='y', labelsize=fs['tick'], which='minor', pad=pad['tick'],)

handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels, loc='upper center', fontsize=fs['legend'], bbox_to_anchor=(0.5, 1.1), ncols=5,
          handlelength=1.5, columnspacing=2, labelspacing=0.5)

dm.simple_layout(fig, bbox=[0, 1, 0, 1], margins=[0.05, 0.05, 0.05, 0.05])
# plt.savefig('figure/HeatPump_model/water_use_schedule_ashpb.svg', dpi=900, transparent=True)
# plt.savefig('figure/HeatPump_model/water_use_schedule_ashpb.png', dpi=900, transparent=True)
dm.save_and_show(fig)


## 5.2 Outdoor air temperature


In [350]:
# ===================== 사용자 입력(수정 지점) =====================
X = time * enex.s2h  # 시간 [h]
Y1 = T_oa  # 실외 공기 온도

X_LABEL = 'Elapsed time [h]'
Y1_LABEL = 'Outdoor air temperature [°C]'

xmin1 = 0
xmax1, xint1, xmar1 = 48, 12, 0
ymin1, ymax1, yint1, ymar1 = -10, 20, 10, 0
color_ax1 = 'oc.blue'
# ===============================================================

fig, ax1 = plt.subplots(1,1, figsize=(dm.cm2in(10), dm.cm2in(5.5)))

ax1.plot(X, Y1, label='Outdoor air temp', linewidth=LW[2], color=color_ax1 + '4', rasterized=True)

ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y1_LABEL, fontsize=fs['label'], labelpad=pad['label'], color=color_ax1 + '6')

ax1.set_xlim(xmin1 - xmar1, xmax1 + xmar1)
ax1.set_ylim(ymin1 - ymar1, ymax1 + ymar1)

ax1.set_xticks(np.arange(xmin1, xmax1*1.001, xint1))
ax1.set_yticks(np.arange(ymin1, ymax1*1.001, yint1))

ax1.spines['left'].set_visible(True)
ax1.spines['left'].set_color(color_ax1 + '6')

ax1.tick_params(labelsize=fs['tick'], which='major', length=2.5, width=0.3, pad=pad['tick'])
ax1.tick_params(labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])

ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

ax1.tick_params(axis='y', colors=color_ax1 + '6', labelsize=fs['tick'], which='major', pad=pad['tick'],)
ax1.tick_params(axis='y', colors=color_ax1 + '6', labelsize=fs['tick'], which='minor', pad=pad['tick'],)

handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels, loc='upper center', fontsize=fs['legend'], bbox_to_anchor=(0.5, 1.1), ncols=5,
          handlelength=1.5, columnspacing=2, labelspacing=0.5)

dm.simple_layout(fig, bbox=[0, 1, 0, 1], margins=[0.05, 0.05, 0.05, 0.05])
# plt.savefig('figure/HeatPump_model/Outdoor_air_temp_ashpb.svg', dpi=900, transparent=True)
# plt.savefig('figure/HeatPump_model/Outdoor_air_temp_ashpb.png', dpi=900, transparent=True)
dm.save_and_show(fig)


## Fig. 1 COP and tank water temp


In [325]:
# ===================== 사용자 입력(수정 지점) =====================
X = time * enex.s2h  # 시간 [h]
Y1 = T_tank_w  # 탱크 온도
Y2 = T_oa      # 옥외 공기 온도
Y3 = COP

X_LABEL = 'Elapsed time [h]'
Y1_LABEL = 'Tank water temp. [°C]'
Y2_LABEL = 'Outdoor air temp. [°C]'
Y3_LABEL = 'COP [ - ]'

xmin1 = 0
xmax1, xint1, xmar1 = xmin1 + 72, 12, 0
ymin1, ymax1, yint1, ymar1 = 0, 80, 20, 2   # 하나의 온도축으로 범위 확장
ymin3, ymax3, yint3 = 1.0, 5.0, 1                                           # COP 축
color_tank = 'oc.pink'
color_ou = 'oc.orange'
color_cop = 'oc.green'
# ===============================================================

fig, ax1 = plt.subplots(1,1, figsize=(dm.cm2in(12), dm.cm2in(5.5)))

# 1st axis: Tank water temperature (라인)
ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel('Temperature [°C]', fontsize=fs['label'], labelpad=pad['label'], color='black')
l1, = ax1.plot(X, Y1, label=Y1_LABEL, linewidth=LW[1], color=color_tank + '4', rasterized=True)
l2, = ax1.plot(X, Y2, label=Y2_LABEL, linewidth=LW[1], color=color_ou + '4', rasterized=True)

ax1.set_xlim(xmin1 - xmar1, xmax1 + xmar1)
ax1.set_ylim(ymin1 - ymar1, ymax1)

ax1.set_xticks(np.arange(xmin1, xmax1*1.001, xint1))
ax1.set_yticks(np.arange(ymin1, ymax1*1.001, yint1))

ax1.spines['left'].set_visible(True)
ax1.spines['left'].set_color('black')

ax1.tick_params(labelsize=fs['tick'], which='major', length=2.5, width=0.3, pad=pad['tick'])
ax1.tick_params(labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])

ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

ax1.tick_params(axis='y', colors='black', labelsize=fs['tick'], which='major', pad=pad['tick'])
ax1.tick_params(axis='y', colors='black', labelsize=fs['tick'], which='minor', pad=pad['tick'])

# 2nd axis: COP
ax2 = ax1.twinx()
mask = Y3 != 0
sct = ax2.scatter(X[mask], Y3[mask], color=color_cop + '4', label=Y3_LABEL, s=0.5, zorder=3,)
ax2.set_ylabel(Y3_LABEL, fontsize=fs['label'], labelpad = pad['label'], color=color_cop + '6')
ax2.set_ylim(ymin3, ymax3)
ax2.set_yticks(np.arange(ymin3, ymax3+1e-6, yint3))
ax2.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

ax2.spines['right'].set_color(color_cop + '6')
ax2.spines['left'].set_visible(False)
ax2.spines['right'].set_visible(True)
ax2.spines['top'].set_visible(False)
ax2.spines['bottom'].set_visible(False)
ax2.tick_params(axis='y', colors=color_cop + '6', labelsize=fs['tick'], which='major', pad=pad['tick'])
ax2.tick_params(axis='y', colors=color_cop + '6', labelsize=fs['tick'], which='minor', pad=pad['tick'])

# Bring (tank/외기)-COP 순으로 zorder 조정
ax1.set_zorder(2)
ax2.set_zorder(1)
ax1.patch.set_alpha(0.0)

# Legend (Y1, Y2는 line, Y3는 마커)
lines = [
    l1, 
    l2, 
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color_cop + '6', markersize=3, label=Y3_LABEL)
]

labels = [Y1_LABEL, Y2_LABEL, Y3_LABEL]
ax1.legend(lines, labels, loc='upper center', fontsize=fs['legend'], bbox_to_anchor=(0.5, 1.1), ncols=3,
          handlelength=1.5, columnspacing=2, labelspacing=0.5)

dm.simple_layout(fig, bbox=[0, 1, 0, 1], margins=[0.05, 0.05, 0.05, 0.05])
# plt.savefig(f'figure/HeatPump_model/Tank_water_temp_&_COP_OutdoorAir_ashpb_{xmin1}.svg', dpi=900, transparent=True)
# plt.savefig(f'figure/HeatPump_model/Tank_water_temp_&_COP_OutdoorAir_ashpb_{xmin1}.png', dpi=900, transparent=True)
dm.save_and_show(fig)


## Fig. 2 Exergy efficiency and tank water temp


In [329]:
# ===================== 사용자 입력(수정 지점) =====================
X = time * enex.s2h  # 시간 [h]
Y1 = T_tank_w  # 탱크 온도
Y2 = T_oa      # 옥외 공기 온도

# 엑서지 효율 계산 (백분율 % 단위)
# Exergy efficiency [%] = (cond_exergy / (E_cmp + E_fan_ou)) * 100
ex_eff = X_ref_cond/E_tot * 100
Y3 = ex_eff

X_LABEL = 'Elapsed time [h]'
Y1_LABEL = 'Tank water temp. [°C]'
Y2_LABEL = 'Outdoor air temp. [°C]'
Y3_LABEL = 'Exergy eff. [%]'

xmin1 = 0
xmax1, xint1, xmar1 = xmin1 + 72, 12, 0
ymin1, ymax1, yint1, ymar1 = 0, 80, 20, 2   # 하나의 온도축으로 범위 확장
ymin3, ymax3, yint3 = 20, 40, 5            # 엑서지 효율 축(0~60% 예상, 필요 시 조정)
color_tank = 'oc.pink'
color_ou = 'oc.orange'
color_exerg = 'oc.violet'
# ===============================================================

fig, ax1 = plt.subplots(1,1, figsize=(dm.cm2in(12), dm.cm2in(5.5)))

# 1st axis: Tank water temperature (라인)
ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel('Temperature [°C]', fontsize=fs['label'], labelpad=pad['label'], color='black')
l1, = ax1.plot(X, Y1, label=Y1_LABEL, linewidth=LW[1], color=color_tank + '4', rasterized=True)
l2, = ax1.plot(X, Y2, label=Y2_LABEL, linewidth=LW[1], color=color_ou + '4', rasterized=True)

ax1.set_xlim(xmin1 - xmar1, xmax1 + xmar1)
ax1.set_ylim(ymin1 - ymar1, ymax1)

ax1.set_xticks(np.arange(xmin1, xmax1*1.001, xint1))
ax1.set_yticks(np.arange(ymin1, ymax1*1.001, yint1))

ax1.spines['left'].set_visible(True)
ax1.spines['left'].set_color('black')

ax1.tick_params(labelsize=fs['tick'], which='major', length=2.5, width=0.3, pad=pad['tick'])
ax1.tick_params(labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])

ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

ax1.tick_params(axis='y', colors='black', labelsize=fs['tick'], which='major', pad=pad['tick'])
ax1.tick_params(axis='y', colors='black', labelsize=fs['tick'], which='minor', pad=pad['tick'])

# 2nd axis: Exergy efficiency
ax2 = ax1.twinx()
mask = Y3 != 0
# 'color'이 문자열 시퀀스일 때 마커 색상 지정 오류 발생 → c로 변경
sct = ax2.scatter(X[mask], Y3[mask], c=color_exerg + '4', label=Y3_LABEL, s=0.5, zorder=3,)  # color → c
ax2.set_ylabel(Y3_LABEL, fontsize=fs['label'], labelpad=pad['label'], color=color_exerg + '6')
ax2.set_ylim(ymin3, ymax3)
ax2.set_yticks(np.arange(ymin3, ymax3+1e-6, yint3))
ax2.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

ax2.spines['right'].set_color(color_exerg + '6')
ax2.spines['left'].set_visible(False)
ax2.spines['right'].set_visible(True)
ax2.spines['top'].set_visible(False)
ax2.spines['bottom'].set_visible(False)
ax2.tick_params(axis='y', colors=color_exerg + '6', labelsize=fs['tick'], which='major', pad=pad['tick'])
ax2.tick_params(axis='y', colors=color_exerg + '6', labelsize=fs['tick'], which='minor', pad=pad['tick'])

# Bring (tank/외기)-Exergy 순으로 zorder 조정
ax1.set_zorder(2)
ax2.set_zorder(1)
ax1.patch.set_alpha(0.0)

# Legend (Y1, Y2는 line, Y3는 마커)
lines = [
    l1,
    l2,
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color_exerg + '6', markersize=3, label=Y3_LABEL)
]

labels = [Y1_LABEL, Y2_LABEL, Y3_LABEL]
ax1.legend(lines, labels, loc='upper center', fontsize=fs['legend'], bbox_to_anchor=(0.5, 1.1), ncols=3,
           handlelength=1.5, columnspacing=2, labelspacing=0.5)

dm.simple_layout(fig, bbox=[0, 1, 0, 1], margins=[0.05, 0.05, 0.05, 0.05])
# plt.savefig(f'figure/HeatPump_model/Tank_water_temp_&_ExergyEff_OutdoorAir_ashpb_{xmin1}.svg', dpi=900, transparent=True)
# plt.savefig(f'figure/HeatPump_model/Tank_water_temp_&_ExergyEff_OutdoorAir_ashpb_{xmin1}.png', dpi=900, transparent=True)
dm.save_and_show(fig)


/tmp/ipykernel_3152647/1361523289.py:8: RuntimeWarning: invalid value encountered in divide
  ex_eff = X_ref_cond/E_tot * 100


In [327]:
ex_eff_no_nan = ex_eff[~np.isnan(ex_eff)]
ex_eff_min = np.min(ex_eff_no_nan)
ex_eff_max = np.max(ex_eff_no_nan)
print(f"엑서지 효율 (nan 제외): min={ex_eff_min:.2f}, max={ex_eff_max:.2f}")

엑서지 효율 (nan 제외): min=18.67, max=33.97


## Fig. 2 Power use


In [328]:
# ===================== 사용자 입력(수정 지점) =====================
X = time * enex.s2h  # 시간 [h]
Y1 = E_cmp * enex.W2kW  # 압축기 전력 [kW]
Y2 = E_fan_ou * enex.W2kW  # 실외기 팬 전력 [kW]

X_LABEL = 'Elapsed time [h]'
Y_LABEL = 'Power use [kW]'

xmin1 = 0
xmax1, xint1, xmar1 = 72, 12, 0
ymin1, ymax1, yint1, ymar1 = 0, 4, 1, 0
colors = ['oc.green', 'oc.orange']
legends = ['Compressor', 'Outdoor fan']
linestyles = ['-', '--']
# ===============================================================

fig, ax1 = plt.subplots(1,1, figsize=(dm.cm2in(10), dm.cm2in(5.5)))

ax1.plot(X, Y1, label=legends[0], linewidth=LW[1], color=colors[0] + '4', linestyle=linestyles[0])
ax1.plot(X, Y2, label=legends[1], linewidth=LW[1], color=colors[1] + '4', linestyle=linestyles[1])

ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y_LABEL, fontsize=fs['label'], labelpad=pad['label'])

ax1.set_xlim(xmin1 - xmar1, xmax1 + xmar1)
ax1.set_ylim(ymin1 - ymar1, ymax1 + ymar1)

ax1.set_xticks(np.arange(xmin1, xmax1*1.001, xint1))
ax1.set_yticks(np.arange(ymin1, ymax1*1.001, yint1))

ax1.tick_params(labelsize=fs['tick'], which='major', length=2.5, width=0.3, pad=pad['tick'])
ax1.tick_params(labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])

ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels, loc='upper center', fontsize=fs['legend'], bbox_to_anchor=(0.5, 1.1), ncols=2,
          handlelength=1.5, columnspacing=2, labelspacing=0.5)

dm.simple_layout(fig, bbox=[0, 1, 0, 1], margins=[0.05, 0.05, 0.05, 0.05])
# plt.savefig('figure/HeatPump_model/Power_consumption_ashpb.svg', dpi=900, transparent=True)
# plt.savefig('figure/HeatPump_model/Power_consumption_ashpb.png', dpi=900, transparent=True)
dm.save_and_show(fig)


## Fig. 3 Fan airflow rates


In [309]:
# ===================== 사용자 입력(수정 지점) =====================
X = time * enex.s2h  # 시간 [h]

# 실내기(Evaporator)의 흡기 온도(T_oa), 토출 공기 온도(T_evap_out)로 가정 (데이터에 맞게 수정 필요)
Y1 = T_oa  # 실내기 흡기 온도 [°C]
Y2 = T_air_ou_out  # 실내기 토출 공기 온도 [°C] (시뮬 결과 포함 필요)

X_LABEL = 'Elapsed time [h]'
Y_LABEL = 'Temperature [°C]'

xmin1 = 0
xmax1, xint1, xmar1 = 48, 12, 0
# y축 최댓값/최솟값 자동 혹은 임의 조정
ymin1, ymax1, yint1, ymar1 = min(np.nanmin(Y1), np.nanmin(Y2)) - 2, max(np.nanmax(Y1), np.nanmax(Y2)) + 2, 5, 0
colors = ['oc.blue', 'oc.red']
legends = ['Inlet air temperature (T_oa)', 'Discharge air temperature (T_evap_out)']
linestyles = ['-', '--']
# ===============================================================

fig, ax1 = plt.subplots(1,1, figsize=(dm.cm2in(10), dm.cm2in(5.5)))

ax1.plot(X, Y1, label=legends[0], linewidth=LW[1], color=colors[0] + '4', linestyle=linestyles[0])
ax1.plot(X, Y2, label=legends[1], linewidth=LW[1], color=colors[1] + '4', linestyle=linestyles[1])

ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y_LABEL, fontsize=fs['label'], labelpad=pad['label'])

ax1.set_xlim(xmin1 - xmar1, xmax1 + xmar1)
ax1.set_ylim(ymin1 - ymar1, ymax1 + ymar1)

ax1.set_xticks(np.arange(xmin1, xmax1*1.001, xint1))
ax1.set_yticks(np.arange(ymin1, ymax1*1.001, yint1))

ax1.tick_params(labelsize=fs['tick'], which='major', length=2.5, width=0.3, pad=pad['tick'])
ax1.tick_params(labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])

ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels, loc='upper center', fontsize=fs['legend'], bbox_to_anchor=(0.5, 1.1), ncols=2,
          handlelength=1.5, columnspacing=2, labelspacing=0.5)

dm.simple_layout(fig, bbox=[0, 1, 0, 1], margins=[0.05, 0.05, 0.05, 0.05])
# plt.savefig('figure/HeatPump_model/Indoor_air_temp_comparison_ashpb.svg', dpi=900, transparent=True)
# plt.savefig('figure/HeatPump_model/Indoor_air_temp_comparison_ashpb.png', dpi=900, transparent=True)
dm.save_and_show(fig)


## Fig. 4 Heat transfer rates


In [310]:
# ===================== 사용자 입력(수정 지점) =====================
X = time * enex.s2h  # 시간 [h]
Y1 = Q_ref_cond * enex.W2kW  # 응축기 열량 [kW]
Y2 = Q_ref_evap * enex.W2kW  # 증발기 열량 [kW] (음수)
# Y3 = Q_cond_load * enex.W2kW  # 저탕조 목표 열량 [kW]   # Tank_load 제거

X_LABEL = 'Elapsed time [h]'
Y_LABEL = 'Heat transfer rate [kW]'

xmin1 = 0
xmax1, xint1, xmar1 = 72, 12, 0
ymin1, ymax1, yint1, ymar1 = 0, 10, 2, 0.5
colors = ['oc.pink', 'oc.blue']  # 'oc.green' 제거
legends = ['Heat rejected by condenser', 'Heat absorbed by evaporator']  # 응축기에서 내보내는 열, 증발기에서 흡수하는 열
linestyles = ['-', '--']
# ===============================================================

fig, ax1 = plt.subplots(1,1, figsize=(dm.cm2in(10), dm.cm2in(5.5)))

ax1.plot(X, Y1, label=legends[0], linewidth=LW[1], color=colors[0] + '3', linestyle=linestyles[0])
ax1.plot(X, Y2, label=legends[1], linewidth=LW[1], color=colors[1] + '4', linestyle=linestyles[1])
# ax1.plot(X, Y3, label=legends[2], linewidth=LW[2], color=colors[2] + '4', linestyle='--', rasterized=True)  # Tank_load 제거
# ax1.axhline(y=0, color='oc.gray3', linestyle='-', linewidth=0.5)

ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y_LABEL, fontsize=fs['label'], labelpad=pad['label'])

ax1.set_xlim(xmin1 - xmar1, xmax1 + xmar1)
ax1.set_ylim(ymin1 - ymar1, ymax1)

ax1.set_xticks(np.arange(xmin1, xmax1*1.001, xint1))
ax1.set_yticks(np.arange(ymin1, ymax1*1.001, yint1))

ax1.tick_params(labelsize=fs['tick'], which='major', length=2.5, width=0.3, pad=pad['tick'])
ax1.tick_params(labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])

ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels, loc='upper center', fontsize=fs['legend'], bbox_to_anchor=(0.5, 1.1), ncols=2,
          handlelength=1.5, columnspacing=2, labelspacing=0.5)  # ncols=2로 변경

dm.simple_layout(fig, bbox=[0, 1, 0, 1], margins=[0.05, 0.05, 0.05, 0.05])
# plt.savefig('figure/HeatPump_model/Heat_transfer_rates_ashpb.svg', dpi=900, transparent=True)
# plt.savefig('figure/HeatPump_model/Heat_transfer_rates_ashpb.png', dpi=900, transparent=True)
dm.save_and_show(fig)


## Fig. 5 Outdoor air temperature and evaporator performance


## E_tot 히트맵: 최적화 변수에 따른 총 전력 소비

최적화 변수 `dT_ref_evap`와 `dT_ref_cond`에 따라 결정되는 `T_evap_K`, `T_cond_K`와 최적화 목적 함수 `E_tot`의 변화를 히트맵으로 시각화합니다.


In [275]:
# 고정 조건 설정
T_tank_w_fixed = 55.0  # [°C]
T_oa_fixed = 20.0      # [°C]
Q_cond_load_fixed = 8000.0  # [W]

# 온도 단위 변환
T_tank_w_K_fixed = enex.calc_util.C2K(T_tank_w_fixed)
T_oa_K_fixed = enex.calc_util.C2K(T_oa_fixed)

# 그리드 생성
dT_ref_evap_range = np.linspace(5.0, 30.0, 100)
dT_ref_cond_range = np.linspace(5.0, 30.0, 100)
dT_ref_evap_grid, dT_ref_cond_grid = np.meshgrid(dT_ref_evap_range, dT_ref_cond_range)

# Calculate T_evap_K and T_cond_K grids
T_evap_K_grid = T_oa_K_fixed - dT_ref_evap_grid
T_cond_K_grid = T_tank_w_K_fixed + dT_ref_cond_grid

# Calculate E_tot, E_cmp, E_fan
print("Calculating power grids...")
E_tot_grid = np.full_like(dT_ref_evap_grid, np.nan)
E_cmp_grid = np.full_like(dT_ref_evap_grid, np.nan)
E_fan_grid = np.full_like(dT_ref_evap_grid, np.nan)

# Initialize constraint grids
cond_constraint_grid = np.full_like(dT_ref_evap_grid, np.nan)
evap_constraint_grid = np.full_like(dT_ref_evap_grid, np.nan)

for i in tqdm(range(len(dT_ref_cond_range)), desc="dT_ref_cond"):
    for j in range(len(dT_ref_evap_range)):
        try:
            result = ashpb_model._calc_on_state(
                optimization_vars=[dT_ref_evap_grid[i, j], dT_ref_cond_grid[i, j]],
                T_tank_w=T_tank_w_fixed,
                Q_cond_load=Q_cond_load_fixed,
                T_oa=T_oa_fixed
            )
            if result is not None and isinstance(result, dict):
                E_tot_grid[i, j] = result['E_tot']
                E_cmp_grid[i, j] = result['E_cmp']
                E_fan_grid[i, j] = result['E_fan_ou']
                
                # Evaluate constraints
                # Condenser constraint: Q_LMTD_cond - Q_cond_load >= 0
                if 'Q_LMTD_cond' in result and not np.isnan(result['Q_LMTD_cond']):
                    cond_constraint_grid[i, j] = result['Q_LMTD_cond'] - Q_cond_load_fixed
                
                # Evaporator constraint: Q_LMTD_evap - Q_ref_evap = 0
                if 'Q_LMTD_evap' in result and 'Q_ref_evap' in result:
                    if not (np.isnan(result['Q_LMTD_evap']) or np.isnan(result['Q_ref_evap'])):
                        evap_constraint_grid[i, j] = result['Q_LMTD_evap'] - result['Q_ref_evap']
        except Exception as e:
            # Keep np.nan for convergence failure or calculation error
            pass

print("Calculation complete!")


Calculating power grids...


dT_ref_cond: 100%|██████████| 100/100 [00:03<00:00, 25.16it/s]

Calculation complete!


In [278]:
# 히트맵 시각화: E_tot, E_cmp, E_fan 분리
fig, axes = plt.subplots(3, 1, figsize=(dm.cm2in(8), dm.cm2in(16)))

# 온도 그리드 (섭씨)
T_evap_C_grid = enex.calc_util.K2C(T_evap_K_grid)
T_cond_C_grid = enex.calc_util.K2C(T_cond_K_grid)

# 히트맵 1: E_tot
im1 = axes[0].imshow(E_tot_grid, aspect='auto', origin='lower', 
                     extent=[dT_ref_evap_range[0], dT_ref_evap_range[-1], 
                            dT_ref_cond_range[0], dT_ref_cond_range[-1]],
                     cmap='viridis', interpolation='nearest')
axes[0].set_xlabel('dT_ref_evap [K]', fontsize=fs['label'])
axes[0].set_ylabel('dT_ref_cond [K]', fontsize=fs['label'])
axes[0].set_title('E_tot [W]', fontsize=fs['subtitle'])
cbar1 = plt.colorbar(im1, ax=axes[0])
cbar1.set_label('E_tot [W]', fontsize=fs['cbar_label'])

# 히트맵 2: E_cmp
im2 = axes[1].imshow(E_cmp_grid, aspect='auto', origin='lower',
                     extent=[dT_ref_evap_range[0], dT_ref_evap_range[-1], 
                            dT_ref_cond_range[0], dT_ref_cond_range[-1]],
                     cmap='plasma', interpolation='nearest')
axes[1].set_xlabel('dT_ref_evap [K]', fontsize=fs['label'])
axes[1].set_ylabel('dT_ref_cond [K]', fontsize=fs['label'])
axes[1].set_title('E_cmp [W]', fontsize=fs['subtitle'])
cbar2 = plt.colorbar(im2, ax=axes[1])
cbar2.set_label('E_cmp [W]', fontsize=fs['cbar_label'])

# 히트맵 3: E_fan
im3 = axes[2].imshow(E_fan_grid, aspect='auto', origin='lower',
                     extent=[dT_ref_evap_range[0], dT_ref_evap_range[-1], 
                            dT_ref_cond_range[0], dT_ref_cond_range[-1]],
                     cmap='coolwarm', interpolation='nearest')
axes[2].set_xlabel('dT_ref_evap [K]', fontsize=fs['label'])
axes[2].set_ylabel('dT_ref_cond [K]', fontsize=fs['label'])
axes[2].set_title('E_fan_ou [W]', fontsize=fs['subtitle'])
cbar3 = plt.colorbar(im3, ax=axes[2])
cbar3.set_label('E_fan_ou [W]', fontsize=fs['cbar_label'])

# 유효한 데이터 포인트 통계 출력
valid_mask = ~np.isnan(E_tot_grid)
n_valid = np.sum(valid_mask)
n_total = E_tot_grid.size
print(f"\n유효한 데이터 포인트: {n_valid}/{n_total} ({100*n_valid/n_total:.1f}%)")
if n_valid > 0:
    print(f"E_tot 범위: {np.nanmin(E_tot_grid):.1f} ~ {np.nanmax(E_tot_grid):.1f} [W]")
    print(f"E_tot 평균: {np.nanmean(E_tot_grid):.1f} [W]")
    print(f"E_cmp 범위: {np.nanmin(E_cmp_grid):.1f} ~ {np.nanmax(E_cmp_grid):.1f} [W]")
    print(f"E_cmp 평균: {np.nanmean(E_cmp_grid):.1f} [W]")
    print(f"E_fan 범위: {np.nanmin(E_fan_grid):.1f} ~ {np.nanmax(E_fan_grid):.1f} [W]")
    print(f"E_fan 평균: {np.nanmean(E_fan_grid):.1f} [W]")

    # E_tot이 제일 작은 조합 찾기 및 출력
    min_idx = np.nanargmin(E_tot_grid)
    min_i, min_j = np.unravel_index(min_idx, E_tot_grid.shape)
    min_dT_ref_evap = dT_ref_evap_grid[min_i, min_j]
    min_dT_ref_cond = dT_ref_cond_grid[min_i, min_j]
    min_E_tot = E_tot_grid[min_i, min_j]
    print(f"\nE_tot 최소값: {min_E_tot:.1f} [W] (dT_ref_evap={min_dT_ref_evap:.2f} K, dT_ref_cond={min_dT_ref_cond:.2f} K)")

plt.tight_layout()
# plt.savefig('figure/HeatPump_model/Power_heatmap.png', dpi=300, transparent=True)
dm.save_and_show(fig)

'''
최적화 결과 정보:
  메시지: Iteration limit reached
  반복 횟수: 1000
  최종 목적 함수 값 (E_tot): 4170.64 [W]
  초기 추정값: dT_ref_evap=20.00 [K], dT_ref_cond=10.01 [K]
  최종 변수 값: dT_ref_evap=6.5333 [K], dT_ref_cond=10.0000 [K]
'''


유효한 데이터 포인트: 4219/10000 (42.2%)
E_tot 범위: 2885.4 ~ 5033.8 [W]
E_tot 평균: 3543.8 [W]
E_cmp 범위: 2220.3 ~ 4746.7 [W]
E_cmp 평균: 3247.5 [W]
E_fan 범위: 35.4 ~ 1558.3 [W]
E_fan 평균: 296.3 [W]

E_tot 최소값: 2885.4 [W] (dT_ref_evap=5.00 K, dT_ref_cond=14.60 K)


'\n최적화 결과 정보:\n  메시지: Iteration limit reached\n  반복 횟수: 1000\n  최종 목적 함수 값 (E_tot): 4170.64 [W]\n  초기 추정값: dT_ref_evap=20.00 [K], dT_ref_cond=10.01 [K]\n  최종 변수 값: dT_ref_evap=6.5333 [K], dT_ref_cond=10.0000 [K]\n'

In [277]:
# Constraint satisfaction region heatmap visualization
print("\nAnalyzing constraints...")

# Evaluate constraint satisfaction
# Condenser constraint: Q_LMTD_cond - Q_cond_load >= 0 (inequality)
cond_satisfied = cond_constraint_grid >= 0

# Evaporator constraint: Q_LMTD_evap - Q_ref_evap = 0 (equality, tolerance: 1.0 W)
evap_tolerance = 1.0  # [W]
evap_satisfied = np.abs(evap_constraint_grid) < evap_tolerance

# Feasible region (both constraints satisfied)
feasible_mask = cond_satisfied & evap_satisfied & ~np.isnan(cond_constraint_grid) & ~np.isnan(evap_constraint_grid)

# Print statistics
n_feasible = np.sum(feasible_mask)
n_valid_constraints = np.sum(~np.isnan(cond_constraint_grid) & ~np.isnan(evap_constraint_grid))
print(f"\nConstraint Statistics:")
print(f"  Valid constraint evaluation points: {n_valid_constraints}/{n_total} ({100*n_valid_constraints/n_total:.1f}%)")
print(f"  Feasible region (both constraints satisfied): {n_feasible}/{n_valid_constraints} ({100*n_feasible/n_valid_constraints:.1f}%)")
print(f"\nCondenser Constraint (Q_LMTD_cond - Q_cond_load):")
if np.sum(~np.isnan(cond_constraint_grid)) > 0:
    print(f"  Range: {np.nanmin(cond_constraint_grid):.2f} ~ {np.nanmax(cond_constraint_grid):.2f} [W]")
    print(f"  Mean: {np.nanmean(cond_constraint_grid):.2f} [W]")
    print(f"  Satisfied points: {np.sum(cond_satisfied)}/{np.sum(~np.isnan(cond_constraint_grid))} ({100*np.sum(cond_satisfied)/np.sum(~np.isnan(cond_constraint_grid)):.1f}%)")
print(f"\nEvaporator Constraint (Q_LMTD_evap - Q_ref_evap):")
if np.sum(~np.isnan(evap_constraint_grid)) > 0:
    print(f"  Range: {np.nanmin(evap_constraint_grid):.2f} ~ {np.nanmax(evap_constraint_grid):.2f} [W]")
    print(f"  Mean: {np.nanmean(evap_constraint_grid):.2f} [W]")
    print(f"  Std dev: {np.nanstd(evap_constraint_grid):.2f} [W]")
    print(f"  Satisfied points (|error| < {evap_tolerance} W): {np.sum(evap_satisfied)}/{np.sum(~np.isnan(evap_constraint_grid))} ({100*np.sum(evap_satisfied)/np.sum(~np.isnan(evap_constraint_grid)):.1f}%)")

# Minimum E_tot in feasible region
if n_feasible > 0:
    feasible_E_tot = E_tot_grid.copy()
    feasible_E_tot[~feasible_mask] = np.nan
    min_feasible_idx = np.nanargmin(feasible_E_tot)
    min_feasible_i, min_feasible_j = np.unravel_index(min_feasible_idx, feasible_E_tot.shape)
    min_feasible_dT_ref_evap = dT_ref_evap_grid[min_feasible_i, min_feasible_j]
    min_feasible_dT_ref_cond = dT_ref_cond_grid[min_feasible_i, min_feasible_j]
    min_feasible_E_tot = feasible_E_tot[min_feasible_i, min_feasible_j]
    print(f"\nMinimum E_tot in feasible region:")
    print(f"  E_tot: {min_feasible_E_tot:.1f} [W]")
    print(f"  dT_ref_evap: {min_feasible_dT_ref_evap:.2f} [K]")
    print(f"  dT_ref_cond: {min_feasible_dT_ref_cond:.2f} [K]")

# Heatmap visualization
fig, axes = plt.subplots(1, 3, figsize=(dm.cm2in(27), dm.cm2in(8)))

# Heatmap 1: Condenser constraint
im1 = axes[0].imshow(cond_constraint_grid, aspect='auto', origin='lower',
                    extent=[dT_ref_evap_range[0], dT_ref_evap_range[-1],
                           dT_ref_cond_range[0], dT_ref_cond_range[-1]],
                    cmap='RdYlGn', interpolation='nearest', vmin=-2000, vmax=2000)
axes[0].set_xlabel('dT_ref_evap [K]', fontsize=fs['label'])
axes[0].set_ylabel('dT_ref_cond [K]', fontsize=fs['label'])
cbar1 = plt.colorbar(im1, ax=axes[0])
cbar1.set_label('Q_LMTD_cond - Q_cond_load [W]', fontsize=fs['cbar_label'])
# Constraint satisfaction boundary (0 line)
axes[0].contour(dT_ref_evap_grid, dT_ref_cond_grid, cond_constraint_grid, 
                levels=[0], colors='black', linewidths=1.5, linestyles='--')
axes[0].text(0.02, 0.98, 'Positive: Satisfied (Green)', transform=axes[0].transAxes,
             fontsize=fs['text'], verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Heatmap 2: Evaporator constraint
im2 = axes[1].imshow(evap_constraint_grid, aspect='auto', origin='lower',
                    extent=[dT_ref_evap_range[0], dT_ref_evap_range[-1],
                           dT_ref_cond_range[0], dT_ref_cond_range[-1]],
                    cmap='RdBu_r', interpolation='nearest', vmin=-100, vmax=100)
axes[1].set_xlabel('dT_ref_evap [K]', fontsize=fs['label'])
axes[1].set_ylabel('dT_ref_cond [K]', fontsize=fs['label'])
cbar2 = plt.colorbar(im2, ax=axes[1])
cbar2.set_label('Q_LMTD_evap - Q_ref_evap [W]', fontsize=fs['cbar_label'])
# Constraint satisfaction boundary (±tolerance)
axes[1].text(0.02, 0.98, f'Close to 0: Satisfied (|error| < {evap_tolerance} W)', transform=axes[1].transAxes,
             fontsize=fs['text'], verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Heatmap 3: Feasible region (both constraints satisfied)
feasible_grid = feasible_mask.astype(float)
feasible_grid[~feasible_mask] = np.nan
im3 = axes[2].imshow(feasible_grid, aspect='auto', origin='lower',
                    extent=[dT_ref_evap_range[0], dT_ref_evap_range[-1],
                           dT_ref_cond_range[0], dT_ref_cond_range[-1]],
                    cmap='RdYlGn', interpolation='nearest', vmin=0, vmax=1)
axes[2].set_xlabel('dT_ref_evap [K]', fontsize=fs['label'])
axes[2].set_ylabel('dT_ref_cond [K]', fontsize=fs['label'])
cbar3 = plt.colorbar(im3, ax=axes[2])
cbar3.set_label('Satisfaction', fontsize=fs['cbar_label'])
cbar3.set_ticks([0, 1])
cbar3.set_ticklabels(['Not Satisfied', 'Satisfied'])

# E_tot contour overlay (on feasible region)
if n_feasible > 0:
    valid_E_tot = E_tot_grid.copy()
    valid_E_tot[~feasible_mask] = np.nan
    if np.sum(~np.isnan(valid_E_tot)) > 0:
        E_tot_levels = np.linspace(np.nanmin(valid_E_tot), np.nanmax(valid_E_tot), 10)
        CS = axes[2].contour(dT_ref_evap_grid, dT_ref_cond_grid, valid_E_tot,
                            levels=E_tot_levels, colors='black', linewidths=0.5, alpha=0.5)
        axes[2].clabel(CS, inline=True, fontsize=fs['text'], fmt='%d')
        # Mark minimum value
        axes[2].plot(min_feasible_dT_ref_evap, min_feasible_dT_ref_cond, 'r*', 
                    markersize=15, label=f'Min E_tot: {min_feasible_E_tot:.1f} W')
        axes[2].legend(fontsize=fs['legend'], loc='upper right')

axes[2].text(0.02, 0.98, f'Feasible region: {n_feasible}/{n_valid_constraints} ({100*n_feasible/n_valid_constraints:.1f}%)',
             transform=axes[2].transAxes, fontsize=fs['text'], verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
# plt.savefig('figure/HeatPump_model/Constraint_heatmap.png', dpi=300, transparent=True)
dm.save_and_show(fig)



Analyzing constraints...

Constraint Statistics:
  Valid constraint evaluation points: 4219/10000 (42.2%)
  Feasible region (both constraints satisfied): 4219/4219 (100.0%)

Condenser Constraint (Q_LMTD_cond - Q_cond_load):
  Range: 6903.62 ~ 28278.36 [W]
  Mean: 17445.18 [W]
  Satisfied points: 4219/4219 (100.0%)

Evaporator Constraint (Q_LMTD_evap - Q_ref_evap):
  Range: -0.00 ~ 0.00 [W]
  Mean: -0.00 [W]
  Std dev: 0.00 [W]
  Satisfied points (|error| < 1.0 W): 4219/4219 (100.0%)

Minimum E_tot in feasible region:
  E_tot: 2885.4 [W]
  dT_ref_evap: 5.00 [K]
  dT_ref_cond: 14.60 [K]
